# Time Series Plots: Before Cleaning vs. After Cleaning

For selected ICU stays, create one figure per vital parameter, each showing a time series plot based on the uncleaned values and one based on the cleaned values.

Accordingly, three figures are generated for each ICU stay:
* Heart rate before and after cleaning
* Blood pressure before and after cleaning
* O2 saturation before and after cleaning

The respective time series are arranged one below the other and share the same x-axis, so that it is easy to compare the time series before and after cleaning.

In [ ]:
import pandas as pd
import pyarrow as pa

# Read from parquet file
# chartevents_clean contains both VALUENUM (values before cleaning) and VALUENUM_CLEAN (values after cleaning)
chartevents_clean = pd.read_parquet('../data/chartevents_clean.parquet', engine='pyarrow')

In [ ]:
# Create set of relevant ICUSTAY_IDs to filter by
icustayid_filter = [282496, 277953, 242623, 221376, 200075, 299742, 294140]
# ICU stay examples used in week 6 presentation: 
# ICU stay 282496 → Heart rate
# ICU stay 277953 → Blood pressure
# ICU stay 242623 → Heart rate
# ICU stay 221376 → Blood pressure
# ICU stay 200075 → O2 saturation
# ICU stay 299742 → O2 saturation
# ICU stay 294140 → Heart rate

# Filter by ITEMIDs
plotdata = chartevents_clean[chartevents_clean.ICUSTAY_ID.isin(icustayid_filter)].copy()

In [ ]:
# Add new column with ITEMID_LABEL, which can be used for the legend of the plot.
import numpy as np
plotdata.insert(loc=len(plotdata.columns), column='ITEMID_LABEL', value=np.nan)
# Heart rate ITEMID_LABELs
plotdata.loc[plotdata['ITEMID'] == 220045, 'ITEMID_LABEL'] = "Heart rate (bpm)"
plotdata.loc[plotdata['ITEMID'] == 220046, 'ITEMID_LABEL'] = "Alarm threshold: High heart rate (bpm)"
plotdata.loc[plotdata['ITEMID'] == 220047, 'ITEMID_LABEL'] = "Alarm threshold: Low heart rate (bpm)"
# Blood pressure ITEMID_LABELs
plotdata.loc[plotdata['ITEMID'] == 220179, 'ITEMID_LABEL'] = "Non-invasive systolic blood pressure (mmHg)"
plotdata.loc[plotdata['ITEMID'] == 223751, 'ITEMID_LABEL'] = "Alarm threshold: High systolic blood pressure (mmHg)"
plotdata.loc[plotdata['ITEMID'] == 223752, 'ITEMID_LABEL'] = "Alarm threshold: Low systolic blood pressure (mmHg)"
# O2 saturation ITEMID_LABELs
plotdata.loc[plotdata['ITEMID'] == 220277, 'ITEMID_LABEL'] = "O2 saturation pulseoxymetry (%)"
plotdata.loc[plotdata['ITEMID'] == 223769, 'ITEMID_LABEL'] = "Alarm threshold: High O2 saturation (%)"
plotdata.loc[plotdata['ITEMID'] == 223770, 'ITEMID_LABEL'] = "Alarm threshold: Low O2 saturation (%)"

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

# Path to folder where the figures will be saved as PNGs
path = '../plots/'

# Set shared variables
sns.set_style("whitegrid")

my_figsize = (12, 10)
my_dpi = 72 # used for display only, DPIs for saved PNGs are set separately

my_suptitle_fontweight = 'bold'
my_suptitle_color = 'black'
my_suptitle_fontsize = 14
my_suptitle_y = 0.95
my_subplots_adjust_hspace = 0.6

my_x = "CHARTTIME"
my_hue = "ITEMID_LABEL"
my_drawstyle = 'steps-post' # Interpolate missing values by using the last available value
my_markers = ['p','^','v']
my_markersize = 5
my_dashes = False
my_palette = [sns.color_palette("colorblind")[0],sns.color_palette("colorblind")[1],sns.color_palette("colorblind")[2]]
fig_title = "Vital parameter histories before and after data cleaning for ICU stay "
xlabel = "Time"

my_legend_loc = 'center left'
my_legend_bbox_to_anchor = (1, 0.5)

my_title_fontweight = 'bold'
my_title_color = 'black'
my_title_fontsize = 13
my_title_y = 1.025
my_title_suffix_before = " before cleaning"
my_title_suffix_after = " after cleaning"

my_xlabel_fontsize = 12
my_xlabel_labelpad = 15
my_ylabel_fontsize = 12
my_ylabel_labelpad = 15
my_xtick_rotation = 90

# Set heart rate variables
hr_title = "Heart rate"
hr_ylabel = "Beats per minute"

# Set blood pressure variables
bp_title = "Non-invasive systolic blood pressure"
bp_ylabel = "Millimeter of mercury"

# Set O2 saturation variables
o2_title = "O2 saturation pulseoxymetry"
o2_ylabel = "Percent"

for icustay in icustayid_filter:

    # Set ICU stay specific variables
    xlimits = [plotdata[plotdata['ICUSTAY_ID'] == icustay].CHARTTIME.min()-pd.Timedelta('1 day'), plotdata[plotdata['ICUSTAY_ID'] == icustay].CHARTTIME.max()+pd.Timedelta('1 day')]

    #########################
    # Heart rate figure
    #########################
    # Config figure
    fig_hr, axs = plt.subplots(2, 1, figsize = my_figsize, dpi = my_dpi)
    fig_hr.suptitle(str(fig_title+str(icustay)), fontweight = my_suptitle_fontweight,color = my_suptitle_color, fontsize = my_suptitle_fontsize, y = my_suptitle_y)
    fig_hr.subplots_adjust(hspace = my_subplots_adjust_hspace)
    # Set heart rate plot variables
    plotdata_hr = plotdata[(plotdata['ITEMID'].isin([220045,220046,220047])) & (plotdata['ICUSTAY_ID'] == icustay)]
    hue_order_hr = [str(plotdata[plotdata['ITEMID'] == 220045].ITEMID_LABEL.iloc[0]),
                     str(plotdata[plotdata['ITEMID'] == 220046].ITEMID_LABEL.iloc[0]),
                     str(plotdata[plotdata['ITEMID'] == 220047].ITEMID_LABEL.iloc[0])]
    # Heart rate before
    ax = sns.lineplot(
        ax = axs[0],
        data = plotdata_hr,
        x = my_x,
        y = "VALUENUM",
        hue = my_hue,
        hue_order = hue_order_hr,
        style = my_hue,
        drawstyle = my_drawstyle,
        markers = my_markers,
        markersize = my_markersize,
        dashes = my_dashes,
        palette = my_palette
        )
    axs[0].set_title(str(hr_title+my_title_suffix_before), fontweight = my_title_fontweight, color = my_title_color, fontsize = my_title_fontsize, y = my_title_y)
    # Heart rate after
    ax = sns.lineplot(
        ax = axs[1],
        data = plotdata_hr,
        x = my_x,
        y = "VALUENUM_CLEAN",
        hue = my_hue,
        hue_order = hue_order_hr,
        style = my_hue,
        drawstyle = my_drawstyle,
        markers = my_markers,
        markersize = my_markersize,
        dashes = my_dashes,
        palette = my_palette
        )
    axs[1].set_title(str(hr_title+my_title_suffix_after), fontweight = my_title_fontweight, color = my_title_color, fontsize = my_title_fontsize, y = my_title_y)
    # Apply formatting for both plots in figure 
    for i, axis in enumerate(axs):
        axs[i].legend(loc = my_legend_loc, bbox_to_anchor = my_legend_bbox_to_anchor)
        axs[i].set_xlim(xlimits)
        axs[i].set_xlabel(xlabel, fontsize = my_xlabel_fontsize, labelpad = my_xlabel_labelpad)
        axs[i].set_ylabel(hr_ylabel, fontsize = my_ylabel_fontsize, labelpad = my_ylabel_labelpad)
        axs[i].tick_params(axis="x", rotation=my_xtick_rotation)
    
    # Save heart rate figure as PNG file
    timestamp = dt.datetime.today().strftime('%Y-%m-%dT%H-%M-%S')
    plt.savefig(str(path + 'icustay-' + str(icustay) + '_before-vs-after-cleaning_'+'hr'+'_72-DPI_' + timestamp + '.png'), dpi=72, bbox_inches='tight')
    plt.savefig(str(path + 'icustay-' + str(icustay) + '_before-vs-after-cleaning_'+'hr'+'_300-DPI_' + timestamp + '.png'), dpi=300, bbox_inches='tight')
    
    # Plot heart rate figure
    plt.show(fig_hr)

    #########################
    # Blood pressure figure
    #########################
    # Config figure
    fig_bp, axs = plt.subplots(2, 1, figsize = my_figsize, dpi = my_dpi)
    fig_bp.suptitle(str(fig_title+str(icustay)), fontweight = my_suptitle_fontweight,color = my_suptitle_color, fontsize = my_suptitle_fontsize, y = my_suptitle_y)
    fig_bp.subplots_adjust(hspace = my_subplots_adjust_hspace)
    # Set heart rate plot variables
    plotdata_bp = plotdata[(plotdata['ITEMID'].isin([220179,223751,223752])) & (plotdata['ICUSTAY_ID'] == icustay)]
    hue_order_bp = [str(plotdata[plotdata['ITEMID'] == 220179].ITEMID_LABEL.iloc[0]),
                    str(plotdata[plotdata['ITEMID'] == 223751].ITEMID_LABEL.iloc[0]),
                    str(plotdata[plotdata['ITEMID'] == 223752].ITEMID_LABEL.iloc[0])]
    # Blood pressure before
    ax = sns.lineplot(
        ax = axs[0],
        data = plotdata_bp,
        x = my_x,
        y = "VALUENUM",
        hue = my_hue,
        hue_order = hue_order_bp,
        style = my_hue,
        drawstyle = my_drawstyle,
        markers = my_markers,
        markersize = my_markersize,
        dashes = my_dashes,
        palette = my_palette
        )
    axs[0].set_title(str(bp_title+my_title_suffix_before), fontweight = my_title_fontweight, color = my_title_color, fontsize = my_title_fontsize, y = my_title_y)
    # Blood pressure after
    ax = sns.lineplot(
        ax = axs[1],
        data = plotdata_bp,
        x = my_x,
        y = "VALUENUM_CLEAN",
        hue = my_hue,
        hue_order = hue_order_bp,
        style = my_hue,
        drawstyle = my_drawstyle,
        markers = my_markers,
        markersize = my_markersize,
        dashes = my_dashes,
        palette = my_palette
        )
    axs[1].set_title(str(bp_title+my_title_suffix_after), fontweight = my_title_fontweight, color = my_title_color, fontsize = my_title_fontsize, y = my_title_y)
    # Apply formatting for both plots in figure 
    for i, axis in enumerate(axs):
        axs[i].legend(loc = my_legend_loc, bbox_to_anchor = my_legend_bbox_to_anchor)
        axs[i].set_xlim(xlimits)
        axs[i].set_xlabel(xlabel, fontsize = my_xlabel_fontsize, labelpad = my_xlabel_labelpad)
        axs[i].set_ylabel(hr_ylabel, fontsize = my_ylabel_fontsize, labelpad = my_ylabel_labelpad)
        axs[i].tick_params(axis="x", rotation=my_xtick_rotation)
    
    # Save blood pressure figure as PNG file
    timestamp = dt.datetime.today().strftime('%Y-%m-%dT%H-%M-%S')
    plt.savefig(str(path + 'icustay-' + str(icustay) + '_before-vs-after-cleaning_'+'bp'+'_72-DPI_' + timestamp + '.png'), dpi=72, bbox_inches='tight')
    plt.savefig(str(path + 'icustay-' + str(icustay) + '_before-vs-after-cleaning_'+'bp'+'_300-DPI_' + timestamp + '.png'), dpi=300, bbox_inches='tight')

    # Plot blood pressure figure
    plt.show(fig_bp)

    #########################
    # O2 saturation figure
    #########################
    # Config figure
    fig_o2, axs = plt.subplots(2, 1, figsize = my_figsize, dpi = my_dpi)
    fig_o2.suptitle(str(fig_title+str(icustay)), fontweight = my_suptitle_fontweight,color = my_suptitle_color, fontsize = my_suptitle_fontsize, y = my_suptitle_y)
    fig_o2.subplots_adjust(hspace = my_subplots_adjust_hspace)
    # Set O2 saturation plot variables
    plotdata_bp = plotdata[(plotdata['ITEMID'].isin([220277,223769,223770])) & (plotdata['ICUSTAY_ID'] == icustay)]
    hue_order_bp = [str(plotdata[plotdata['ITEMID'] == 220277].ITEMID_LABEL.iloc[0]),
                    str(plotdata[plotdata['ITEMID'] == 223769].ITEMID_LABEL.iloc[0]),
                    str(plotdata[plotdata['ITEMID'] == 223770].ITEMID_LABEL.iloc[0])]
    # O2 saturation before
    ax = sns.lineplot(
        ax = axs[0],
        data = plotdata_bp,
        x = my_x,
        y = "VALUENUM",
        hue = my_hue,
        hue_order = hue_order_bp,
        style = my_hue,
        drawstyle = my_drawstyle,
        markers = my_markers,
        markersize = my_markersize,
        dashes = my_dashes,
        palette = my_palette
        )
    axs[0].set_title(str(o2_title+my_title_suffix_before), fontweight = my_title_fontweight, color = my_title_color, fontsize = my_title_fontsize, y = my_title_y)
    # O2 saturation after
    ax = sns.lineplot(
        ax = axs[1],
        data = plotdata_bp,
        x = my_x,
        y = "VALUENUM_CLEAN",
        hue = my_hue,
        hue_order = hue_order_bp,
        style = my_hue,
        drawstyle = my_drawstyle,
        markers = my_markers,
        markersize = my_markersize,
        dashes = my_dashes,
        palette = my_palette
        )
    axs[1].set_title(str(o2_title+my_title_suffix_after), fontweight = my_title_fontweight, color = my_title_color, fontsize = my_title_fontsize, y = my_title_y)
    # Apply formatting for both plots in figure 
    for i, axis in enumerate(axs):
        axs[i].legend(loc = my_legend_loc, bbox_to_anchor = my_legend_bbox_to_anchor)
        axs[i].set_xlim(xlimits)
        axs[i].set_xlabel(xlabel, fontsize = my_xlabel_fontsize, labelpad = my_xlabel_labelpad)
        axs[i].set_ylabel(hr_ylabel, fontsize = my_ylabel_fontsize, labelpad = my_ylabel_labelpad)
        axs[i].tick_params(axis="x", rotation=my_xtick_rotation)
    
    # Save O2 saturation figure as PNG file
    timestamp = dt.datetime.today().strftime('%Y-%m-%dT%H-%M-%S')
    plt.savefig(str(path + 'icustay-' + str(icustay) + '_before-vs-after-cleaning_'+'o2'+'_72-DPI_' + timestamp + '.png'), dpi=72, bbox_inches='tight')
    plt.savefig(str(path + 'icustay-' + str(icustay) + '_before-vs-after-cleaning_'+'o2'+'_300-DPI_' + timestamp + '.png'), dpi=300, bbox_inches='tight')
    
    # Plot O2 saturation figure
    plt.show(fig_o2)
    
